In [46]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# Инициализация браузера
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Имя файла для записи
output_file = "responses_output.txt"

try:
    # Базовый URL
    base_url = 'https://www.banki.ru/services/responses/bank/domrfbank/product/hypothec/?is_countable=on&page='
    page = 1

    # Открываем файл для записи
    with open(output_file, "w", encoding="utf-8") as file:
        while True:
            # Переходим на текущую страницу
            driver.get(base_url + str(page))
            print(f"Открываем страницу {page}")

            try:
                # Дожидаемся загрузки основного блока отзывов
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.Responses__StyledResponsesWrapper-sc-1q4fm3k-0.dZUylm'))
                )

                # Находим список отзывов на текущей странице
                wrapper_div = driver.find_element(By.CSS_SELECTOR, 'div.Responses__StyledResponsesWrapper-sc-1q4fm3k-0.dZUylm')
                list_div = wrapper_div.find_element(By.CSS_SELECTOR, 'div.Responsesstyled__StyledList-sc-150koqm-5.ca-dtrb')
                response_elements = list_div.find_elements(By.CSS_SELECTOR, 'div[data-test="responses__response"]')

                # Если нет отзывов, прерываем цикл
                if not response_elements:
                    print("Больше отзывов нет.")
                    file.write("Больше отзывов нет.\n")
                    break

                # Извлекаем текст из отзывов на текущей странице
                for index, element in enumerate(response_elements, start=1):
                    response_text = element.text
                    print(f"Отзыв {index}: {response_text}")
                    file.write(f"Страница {page}, отзыв {index}: {response_text}\n")

            except Exception as e:
                print(f"Ошибка на странице {page}: {e}")
                file.write(f"Ошибка на странице {page}: {e}\n")
                break

            # Переходим на следующую страницу
            page += 1
            time.sleep(2)  # Задержка для имитации действий пользователя

except Exception as e:
    print(f"Глобальная ошибка: {e}")

finally:
    # Закрываем браузер
    driver.quit()
    print(f"Результаты записаны в файл {output_file}")


Открываем страницу 1
Отзыв 1: Хороший банк
Оценка:
5
Зачтено
Являюсь клиентом с 2023 года. Ранее взяли ипотеку в этом банке. Прошел год, необходимо продлить страхование , куда обращаться не знали. Пришли в банк, очереди не было, попросили взять талон. Ожидали 2 минуты, приняла сотрудник Валентина, она...
04.12.2024 10:53
1
Отзыв проверен
Ответ банка
Отзыв 2: Сделка по ипотеке
Оценка:
5
Зачтено
Планировали купить жилой дом по программе семейная ипотека, заявку оформили в дом РФ, так же сделка осложнялась с форматом работы супруга, т.к. у него удаленная работа. При этом сотрудники офиса и банка активно помогали в возможности...
04.12.2024 09:04
2
Отзыв проверен
Ответ банка
Отзыв 3: Онлайн ипотека
Оценка:
5
Зачтено
Давно мечтала обеспечить дочку квартирой в Москве. И в этом мне помогли очень ответственные сотрудники банка Дом РФ- С-ва Юлия и Х-ва Ольга. они помогли мне со сбором необходимых документов, быстро согласовали объект покупки, ставку по госпрограмме...
27.11.2024 20:56
Документы

In [65]:
import csv
import re

# Имя входного текстового файла
input_file = "responses_output.txt"

# Имя выходного CSV файла
output_csv = "responses_output.csv"

# Шаблоны регулярных выражений
review_pattern = re.compile(r"^Страница \d+, отзыв \d+: (.+)$")  # Захватываем заголовок
rating_pattern = re.compile(r"^Оценка:\s*(\d+)$")               # Захватываем только число оценки
date_pattern = re.compile(r"^\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}$")  # Захватываем дату

# Читаем текстовый файл и преобразуем его в CSV
with open(input_file, "r", encoding="utf-8") as infile, open(output_csv, "w", encoding="utf-8", newline='') as outfile:
    csv_writer = csv.writer(outfile)

    # Записываем заголовки
    csv_writer.writerow(["Review Title", "Rating", "Review Text", "Date"])

    # Переменные для хранения текущего отзыва
    review_title = None
    rating = None
    review_text = ""
    date = None

    for line in infile:
        line = line.strip()

        # Пропускаем пустые строки
        if not line:
            continue

        # Ищем заголовок отзыва
        review_match = review_pattern.match(line)
        if review_match:
            # Сохраняем предыдущий отзыв перед обработкой нового
            if review_title is not None:
                csv_writer.writerow([review_title, rating, review_text.strip(), date])
                review_text = ""
                rating = None
                date = None

            review_title = review_match.group(1)
            continue

        # Ищем оценку
        rating_match = rating_pattern.match(line)
        if rating_match:
            rating = rating_match.group(1)  # Только число оценки
            continue

        # Ищем дату
        if date_pattern.match(line):
            date = line
            continue

        # Исключаем строки, которые не нужны
        if line in ["Зачтено", "1", "2", "Отзыв проверен", "Ответ банка"]:
            continue

        # Считаем остальной текст частью отзыва
        review_text += line + " "

    # Записываем последний отзыв в CSV
    if review_title is not None:
        csv_writer.writerow([review_title, rating, review_text.strip(), date])

print(f"Данные успешно сохранены в {output_csv}")




Данные успешно сохранены в responses_output.csv


In [2]:
import pandas as pd
import re

# Путь к входному CSV-файлу
input_csv = "responses_output_corrected.csv"

# Чтение файла
df = pd.read_csv(input_csv, header=None, names=["Review Title", "Rating", "Review Text", "Date"])

# Коррекция поля Rating и разделение Review Text
def correct_row(row):
    if row["Review Text"].startswith("Оценка:"):
        match = re.match(r"Оценка:\s*(\d+)\s*(.+)", row["Review Text"])
        if match:
            row["Rating"] = match.group(1)
            row["Review Text"] = match.group(2)
    return row

# Применение коррекции ко всем строкам
df = df.apply(correct_row, axis=1)

# Сохранение в новый CSV-файл
output_csv = "responses_output_corrected.csv"
df.to_csv(output_csv, index=False, header=False, quoting=1)  # quoting=1 для кавычек в тексте

output_csv


'responses_output_corrected.csv'

In [3]:
import pandas as pd

# Укажите путь к вашему существующему CSV-файлу
input_csv = "responses_output_corrected.csv"

# Укажите путь для сохранения нового файла с ID
output_csv = "reviews_output_with_id.csv"

# Чтение CSV
df = pd.read_csv(input_csv)

# Добавление столбца ID с последовательной нумерацией, начиная с 1
df.insert(0, "ID", range(1, len(df) + 1))

# Сохранение в новый CSV
df.to_csv(output_csv, index=False)

print(f"Файл с добавленным столбцом ID сохранён как {output_csv}")


Файл с добавленным столбцом ID сохранён как reviews_output_with_id.csv
